In [ ]:
import os
import glob
from osgeo import osr,gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# !pip install ipython-autotime
# %load_ext autotime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
photon_df = pd.read_csv("/content/drive/My Drive/PS/PS Data/Badrinath/final.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
ds = gdal.Open("/content/drive/My Drive/PS/PS Data/Badrinath/Badrinath_SRTMGL1Ellip.tif")
transform = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
 
#plt.imshow(array)
array

array([[5513.8896, 5518.893 , 5483.8965, ..., 5034.974 , 5032.974 ,
        5030.9736],
       [5495.89  , 5506.894 , 5494.8975, ..., 5032.976 , 5033.976 ,
        5034.9756],
       [5483.891 , 5496.8945, 5507.898 , ..., 5028.978 , 5030.978 ,
        5037.9775],
       ...,
       [2727.66  , 2717.6643, 2705.669 , ..., 3450.6377, 3419.6375,
        3394.6375],
       [2711.6582, 2698.6626, 2681.6672, ..., 3424.6367, 3400.6367,
        3389.6367],
       [2698.6565, 2666.6611, 2658.6655, ..., 3408.636 , 3390.636 ,
        3374.6357]], dtype=float32)

In [ ]:
width = ds.RasterXSize
height = ds.RasterYSize
print(width)
print(height)

1196
771


In [ ]:
gt = ds.GetGeoTransform()

In [ ]:
origin_x = gt[0]
origin_y = gt[3]
pixel_res_x = gt[1]
pixel_res_y = gt[5]

In [ ]:
def latitude(n):
    return origin_y + (n/width * pixel_res_y)
def longitude(n):
    return origin_x + (n%width * pixel_res_x)

In [ ]:
array = np.array(array)
array = array.flatten()
dem_df = pd.DataFrame(data = array,columns = ['dem height'])
dem_df['index1']=dem_df.index
dem_df['latitude'] = dem_df.index1.apply(latitude)
dem_df['longitude'] = dem_df.index1.apply(longitude)
dem_df = dem_df.drop(['index1'],axis=1)
dem_df = dem_df[['latitude','longitude','dem height']]

In [ ]:
dem_df

,latitude,longitude,dem height
0,30.880694,79.206806,5513.889648
1,30.880694,79.207083,5518.893066
2,30.880694,79.207361,5483.896484
3,30.880694,79.207639,5457.899902
4,30.880694,79.207917,5441.903809
...,...,...,...
922111,30.666529,79.537639,3450.636230
922112,30.666529,79.537917,3434.635986
922113,30.666528,79.538194,3408.635986
922114,30.666528,79.538472,3390.635986


In [ ]:
def get_latitude_index(lat):
  return int((lat-origin_y)/pixel_res_y)

def get_longitude_index(lon):
  return int((lon-origin_x)/pixel_res_x)

def get_high_res_latitude_index(lat):
  return int((lat-origin_y)/(pixel_res_y/3))

def get_high_res_longitude_index(lon):
  return int((lon-origin_x)/(pixel_res_x/3))

In [ ]:
photon_df['latitude_index']= photon_df.latitude.apply(get_latitude_index)
photon_df['longitude_index']= photon_df.longitude.apply(get_longitude_index)
photon_df['latitude_index2']= photon_df.latitude.apply(get_high_res_latitude_index)
photon_df['longitude_index2']= photon_df.longitude.apply(get_high_res_longitude_index)

In [ ]:
photon_df['container_box_id']=(photon_df['latitude_index'])*(width)+photon_df['longitude_index']

In [ ]:
photon_df['container_box_id2']=(photon_df['latitude_index2'])*(width*3)+photon_df['longitude_index2']

In [ ]:
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id,container_box_id2
index,,,,,,,,,
0,30.687405,79.191785,4369.1890,695,-54,2087,-162,831166,7487994
1,30.687405,79.191786,4375.3545,695,-54,2087,-162,831166,7487994
2,30.687405,79.191786,4372.4824,695,-54,2087,-162,831166,7487994
3,30.687405,79.191786,4371.6934,695,-54,2087,-162,831166,7487994
4,30.687406,79.191785,4368.3300,695,-54,2087,-162,831166,7487994
...,...,...,...,...,...,...,...,...,...
1336999,30.863483,79.204341,5573.2075,61,-8,185,-26,72948,663754
1337000,30.863483,79.204341,5572.8440,61,-8,185,-26,72948,663754
1337001,30.863483,79.204341,5573.1230,61,-8,185,-26,72948,663754


In [ ]:
photon_df=photon_df.groupby(['container_box_id2']).mean()
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id
container_box_id2,,,,,,,,
663483,30.863485,79.179274,5934.822300,61.0,-99.0,185.0,-297.0,72857.0
663578,30.863491,79.188049,5329.479165,61.0,-67.0,185.0,-202.0,72889.0
663754,30.863479,79.204328,5572.857538,61.0,-8.0,185.0,-26.0,72948.0
663763,30.863501,79.205182,5570.853552,61.0,-5.0,185.0,-17.0,72951.0
664111,30.863502,79.237513,5650.127409,61.0,110.0,185.0,331.0,73066.0
...,...,...,...,...,...,...,...,...
7494395,30.687314,79.452305,3569.479627,696.0,883.0,2088.0,2651.0,833299.0
7494438,30.687329,79.456318,3488.653082,696.0,898.0,2088.0,2694.0,833314.0
7494740,30.687341,79.484221,2769.532633,696.0,998.0,2088.0,2996.0,833414.0


In [ ]:
photon_df=photon_df.drop(['latitude_index'],axis=1)
photon_df=photon_df.drop(['longitude_index'],axis=1)
photon_df=photon_df.drop(['latitude_index2'],axis=1)
photon_df=photon_df.drop(['longitude_index2'],axis=1)
photon_df.reset_index(drop=True, inplace=True)

In [ ]:
photon_df["container_box_id"] = photon_df["container_box_id"].astype(int)

In [ ]:
photon_df

,latitude,longitude,photon height,container_box_id
0,30.863485,79.179274,5934.822300,72857
1,30.863491,79.188049,5329.479165,72889
2,30.863479,79.204328,5572.857538,72948
3,30.863501,79.205182,5570.853552,72951
4,30.863502,79.237513,5650.127409,73066
...,...,...,...,...
81703,30.687314,79.452305,3569.479627,833299
81704,30.687329,79.456318,3488.653082,833314
81705,30.687341,79.484221,2769.532633,833414
81706,30.687324,79.484659,2762.454950,833416


In [ ]:
merged_df = photon_df.merge(dem_df, left_on='container_box_id', right_index = True)

In [ ]:
merged_df

,latitude_x,longitude_x,photon height,container_box_id,latitude_y,longitude_y,dem height
0,30.863485,79.179274,5934.822300,72857,30.863773,79.511528,4886.112793
1,30.863491,79.188049,5329.479165,72889,30.863766,79.520417,5079.104980
2,30.863479,79.204328,5572.857538,72948,30.863752,79.536806,5741.090332
3,30.863501,79.205182,5570.853552,72951,30.863751,79.537639,5756.089844
4,30.863502,79.237513,5650.127409,73066,30.863724,79.237361,5649.315918
...,...,...,...,...,...,...,...
81702,30.687326,79.452247,3571.340833,833299,30.687156,79.452083,3566.280518
81703,30.687314,79.452305,3569.479627,833299,30.687156,79.452083,3566.280518
81704,30.687329,79.456318,3488.653082,833314,30.687153,79.456250,3493.317627
81705,30.687341,79.484221,2769.532633,833414,30.687129,79.484028,2769.565918


In [ ]:
merged_df.drop(columns = ['container_box_id'], inplace = True)
X = merged_df.drop('photon height',axis=1)
y = merged_df['photon height']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train= scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [ ]:
X_train.shape

(57195, 5)

In [ ]:
X_test.shape

(24513, 5)

In [ ]:
type(X_test)

numpy.ndarray

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score


In [ ]:
model = Ridge()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 2, 0.02)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, y)
# summarize
print('Config: %s' % results.best_params_)

Config: {'alpha': 0.0}


In [ ]:
model=Ridge(alpha=results.best_params_['alpha'])
model.fit(X_train,y_train)


Ridge(alpha=0.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
finalarray=np.zeros([height*3*width*3], dtype = int)
df= pd.DataFrame(columns=['zero'],data=finalarray)
df

,zero
0,0
1,0
2,0
3,0
4,0
...,...
8299039,0
8299040,0
8299041,0
8299042,0


In [ ]:
def latitude_final(n):
    return origin_y + (n/(width*3) * pixel_res_y/3.0)
def longitude_final(n):
    return origin_x + (n%(width*3) * pixel_res_x/3.0)

In [ ]:
df['index1']=df.index
df['latitude']=df.index1.apply(latitude_final)
df['longitude']=df.index1.apply(longitude_final)

In [ ]:
df=df.drop(columns=['index1','zero'],axis=1)
df['latitude_index']= df.latitude.apply(get_latitude_index)
df['longitude_index']= df.longitude.apply(get_longitude_index)
df['container_box_id']=(df['latitude_index'])*(width)+df['longitude_index']
df=df.drop(['latitude_index'],axis=1)
df=df.drop(['longitude_index'],axis=1)
df["container_box_id"] = df["container_box_id"].astype(int)

In [ ]:
final_merged_df = df.merge(dem_df, left_on='container_box_id', right_index = True)
final_merged_df.drop(columns = ['container_box_id'], inplace = True)

In [ ]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,30.880694,79.206806,30.880694,79.206806,5513.889648
1,30.880694,79.206898,30.880694,79.206806,5513.889648
2,30.880694,79.206991,30.880694,79.206806,5513.889648
3588,30.880602,79.206806,30.880694,79.206806,5513.889648
3589,30.880602,79.206898,30.880694,79.206806,5513.889648
...,...,...,...,...,...
8291867,30.666713,79.538935,30.666528,79.538750,3374.635742
8295454,30.666620,79.538843,30.666528,79.538750,3374.635742
8295455,30.666620,79.538935,30.666528,79.538750,3374.635742
8299042,30.666528,79.538843,30.666528,79.538750,3374.635742


In [ ]:
final_merged_df.sort_index()

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,30.880694,79.206806,30.880694,79.206806,5513.889648
1,30.880694,79.206898,30.880694,79.206806,5513.889648
2,30.880694,79.206991,30.880694,79.206806,5513.889648
3,30.880694,79.207083,30.880694,79.207083,5518.893066
4,30.880694,79.207176,30.880694,79.207083,5518.893066
...,...,...,...,...,...
8299039,30.666528,79.538565,30.666528,79.538472,3390.635986
8299040,30.666528,79.538657,30.666528,79.538472,3390.635986
8299041,30.666528,79.538750,30.666528,79.538472,3390.635986
8299042,30.666528,79.538843,30.666528,79.538750,3374.635742


In [ ]:
final_merged_df.head(6963)

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,30.880694,79.206806,30.880694,79.206806,5513.889648
1,30.880694,79.206898,30.880694,79.206806,5513.889648
2,30.880694,79.206991,30.880694,79.206806,5513.889648
3588,30.880602,79.206806,30.880694,79.206806,5513.889648
3589,30.880602,79.206898,30.880694,79.206806,5513.889648
...,...,...,...,...,...
9494,30.880449,79.421435,30.880515,79.421250,4779.511230
9495,30.880449,79.421528,30.880515,79.421250,4779.511230
2320,30.880635,79.421620,30.880515,79.421528,4780.512695
2321,30.880635,79.421713,30.880515,79.421528,4780.512695


In [ ]:
X = final_merged_df
X = scaler_x.transform(X)

In [ ]:
predictions = model.predict(X)
final_merged_df['predictions']=predictions
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,predictions
0,30.880694,79.206806,30.880694,79.206806,5513.889648,0.617374
1,30.880694,79.206898,30.880694,79.206806,5513.889648,0.617311
2,30.880694,79.206991,30.880694,79.206806,5513.889648,0.617247
3588,30.880602,79.206806,30.880694,79.206806,5513.889648,0.617241
3589,30.880602,79.206898,30.880694,79.206806,5513.889648,0.617178
...,...,...,...,...,...,...
8291867,30.666713,79.538935,30.666528,79.538750,3374.635742,0.130349
8295454,30.666620,79.538843,30.666528,79.538750,3374.635742,0.130279
8295455,30.666620,79.538935,30.666528,79.538750,3374.635742,0.130216
8299042,30.666528,79.538843,30.666528,79.538750,3374.635742,0.130146


In [ ]:
final_merged_df['height']=scaler_y.inverse_transform(final_merged_df['predictions'].values.reshape(-1, 1))

In [ ]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,predictions,height
0,30.880694,79.206806,30.880694,79.206806,5513.889648,0.617374,5615.386517
1,30.880694,79.206898,30.880694,79.206806,5513.889648,0.617311,5615.091362
2,30.880694,79.206991,30.880694,79.206806,5513.889648,0.617247,5614.796207
3588,30.880602,79.206806,30.880694,79.206806,5513.889648,0.617241,5614.770564
3589,30.880602,79.206898,30.880694,79.206806,5513.889648,0.617178,5614.475409
...,...,...,...,...,...,...,...
8291867,30.666713,79.538935,30.666528,79.538750,3374.635742,0.130349,3359.251419
8295454,30.666620,79.538843,30.666528,79.538750,3374.635742,0.130279,3358.930621
8295455,30.666620,79.538935,30.666528,79.538750,3374.635742,0.130216,3358.635466
8299042,30.666528,79.538843,30.666528,79.538750,3374.635742,0.130146,3358.314668


In [ ]:
final_merged_df=final_merged_df.sort_index()

In [ ]:
# from google.colab import files

# df.to_csv('final_merged_df.csv')
# files.download('final_merged_df.csv')

In [ ]:
final_merged_df=final_merged_df.drop(columns=['latitude_x','longitude_x','latitude_y','longitude_y','predictions','dem height'])

In [ ]:
final_merged_df

,height
0,5615.386517
1,5615.091362
2,5614.796207
3,5620.012935
4,5619.717779
...,...
8299039,3373.613814
8299040,3373.318659
8299041,3373.023503
8299042,3358.314668


In [ ]:
finalarray = final_merged_df.to_numpy()
finalarray=finalarray.reshape(height*3 ,width*3)
finalarray

array([[5615.38651713, 5615.09136203, 5614.79620694, ..., 5014.77601481,
        5013.34834035, 5013.05318526],
       [5614.77056405, 5614.47540895, 5614.18025386, ..., 5014.16006173,
        5012.73238727, 5012.43723218],
       [5614.15461097, 5613.85945588, 5613.56430078, ..., 5013.54410865,
        5012.11643419, 5011.8212791 ],
       ...,
       [2872.76813416, 2861.87937551, 2861.58422042, ..., 3374.25540961,
        3359.54657407, 3359.25141897],
       [2861.55857753, 2861.26342244, 2860.96826734, ..., 3373.63945653,
        3358.93062099, 3358.63546589],
       [2860.94262445, 2860.64746936, 2860.35231426, ..., 3373.02350345,
        3358.31466791, 3358.01951281]])

In [ ]:
dst_filename = 'Badrinath2ridgeregression.tiff'
x_pixels = width*3  # number of pixels in x
y_pixels = height*3  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(finalarray)

0

In [ ]:
gt

(79.20680555558994,
 0.00027777777777781464,
 0.0,
 30.880694444440582,
 0.0,
 -0.00027777777777781464)

In [ ]:
gtlist = list(gt)
gtlist[1]=gtlist[1]/3
gtlist[5]=gtlist[5]/3
gt=tuple(gtlist)

In [ ]:
geotrans = gt  #get GeoTranform from existed 'data0'
proj=ds.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [ ]:
ls

Badrinath2ridgeregression.tiff  sikkim2linearregression.tiff
drive/                          udaipur2ridgeregression.tiff
sample_data/


In [ ]:
from google.colab import files
files.download('Badrinath2ridgeregression.tiff')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>